In [1]:
import os
from data_handler.models import Asset, Source, Article
from datetime import datetime
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
import pandas as pd
from sentiment.models import Category, Label
from sentiment.model import SentimentPriceModel
import gensim
from data_handler.helpers import progress
from nltk.corpus import stopwords
from scipy.stats import pearsonr
import numpy as np

In [66]:
# Assets
spm = SentimentPriceModel()
spm.add_asset_variable(asset="^FTMC", column_name="FTSE 250", zscore=True, volume=True, get_asset=False)
spm.add_asset_variable(asset="^ISEQ", column_name="ISEQ All Share", zscore=True, volume=True, get_asset=False)
spm.add_asset_variable(asset="^STOXX50E", column_name="Euro Stoxx 50", zscore=True, volume=True, get_asset=True)

,FTSE 250 Volume,ISEQ All Share Volume,Euro Stoxx 50 Return,Euro Stoxx 50 Volume
date,,,,
2016-01-04,-0.846024,0.016210,-3.632469,0.792579
2016-01-05,-0.936379,-0.249442,0.168752,-0.150276
2016-01-06,-0.403054,-0.011577,-1.150151,0.503741
2016-01-07,-0.300610,0.617958,-1.703234,1.891533
2016-01-08,-0.864265,0.633654,-2.036661,0.878035
...,...,...,...,...
2019-12-23,-1.543409,-0.709089,0.041056,-1.196817
2019-12-24,-2.746226,-0.926064,0.000513,-1.632069
2019-12-27,-1.817123,-0.851376,0.158570,-1.118198


In [4]:
# Sources
uk_sources = list(Source.objects.filter(country=1).values_list('id', flat=True))
irish_sources = list(Source.objects.filter(country=0).values_list('id', flat=True))

In [67]:
source_signals = 0
for s in uk_sources:
    sourcename=Source.objects.get(id=s).name
    
    if not "Guardian" in sourcename:
        continue
    spm.add_sentiment_variable(column_name="{}".format(sourcename), sentiwordnet=True, include_pos = False,source_filter=[s],set=True)
if source_signals == 0:
    df = spm.get_df()
    df["UK Negative"] = 0
    for c in uk_sources:
        sourcename=Source.objects.get(id=c).name
        df["UK Negative"] += df["{} Negative".format(sourcename)]
        df = df.drop(["{} Negative".format(sourcename)], axis=1)
    spm.multivariate_df = df

[]
Getting articles without headline
12942
<QuerySet [6]>
Getting zscore===============================================] 100.0% ...2019-12-31
Zscore of The Guardian(London) Negative


KeyError: 'Financial Times (London, England) Negative'

In [20]:
source_signals = 0
for s in uk_sources:
    sourcename=Source.objects.get(id=s).name
    spm.add_sentiment_variable(column_name="{}".format(sourcename), sentiwordnet=True, h_contents = ["market", "economy", "trade"], include_pos = False,source_filter=[s],set=True)
if source_signals == 0:
    df = spm.get_df()
    df["UK Negative"] = 0
    for c in uk_sources:
        sourcename=Source.objects.get(id=c).name
        df["UK Negative"] += df["{} Negative".format(sourcename)]
        df = df.drop(["{} Negative".format(sourcename)], axis=1)
    spm.multivariate_df = df

[]
4216
Multiplying by weights: The Irish Times
Getting zscore
Zscore of The Irish Times


ZeroDivisionError: division by zero

In [12]:
spm = SentimentPriceModel()
spm.load_csv("swn_vartest_2.csv")

,FTSE 250 Volume,ISEQ All Share Volume,Euro Stoxx 50 Return,Euro Stoxx 50 Volume,The Guardian(London) Negative
date,,,,,
2016-01-04,-0.846024,0.016210,-3.632469,0.792579,NaN
2016-01-05,-0.936379,-0.249442,0.168752,-0.150276,NaN
2016-01-06,-0.403054,-0.011577,-1.150151,0.503741,NaN
2016-01-07,-0.300610,0.617958,-1.703234,1.891533,NaN
2016-01-08,-0.864265,0.633654,-2.036661,0.878035,NaN
...,...,...,...,...,...
2019-12-23,-1.543409,-0.709089,0.041056,-1.196817,-1.019529
2019-12-24,-2.746226,-0.926064,0.000513,-1.632069,-0.940543
2019-12-27,-1.817123,-0.851376,0.158570,-1.118198,-0.465974


In [ ]:
spm.multivariate_df.shape
def adjust(val, length= 6): return str(val).ljust(length)
from statsmodels.tsa.vector_ar.vecm import coint_johansen

def cointegration_test(df, key_col, alpha=0.05): 
    """Perform Johanson's Cointegration Test and Report Summary"""
    out = coint_johansen(df,-1,5)
    d = {'0.90':0, '0.95':1, '0.99':2}
    traces = out.lr1
    cvts = out.cvt[:, d[str(1-alpha)]]
    print('\nName   ::  Test Stat > C(95%)    =>   Signif  \n', '--'*20)
    for col, trace, cvt in zip(df.columns, traces, cvts):
        if col == key_col:
            print(adjust(col), ':: ', adjust(round(trace,2), 9), ">", adjust(cvt, 8), ' =>  ' , trace > cvt)

In [10]:
spm.multivariate_df

AttributeError: 'SentimentPriceModel' object has no attribute 'multivariate_df'

In [273]:
order_table = spm.var().model.select_order(15).summary()

/home/kate/final year project/sentiment_analysis/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:218: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.



In [10]:
from statsmodels.tsa.stattools import acf

In [13]:
key_col = 'UK Negative'
df = pd.DataFrame()
spm2 = SentimentPriceModel()
mv = spm.get_df()
cols = mv.columns  
acc_df = pd.DataFrame()
acc_df[key_col] = mv[key_col]
acc_df.set_index(mv.index)
counter = 1
stop = 6
lag = 5

for col in cols:
    if col == key_col:
        continue
    model = "Model {}".format(counter)
    acc_df[col] = mv[col]
    acc_df = acc_df.dropna()
    spm2.set_df(acc_df)
    var = spm2.var()
    fit = var.model.fit(lag)
    index = fit.get_eq_index(key_col)
    df_out = pd.DataFrame(columns = [model])
    lookup = {}
    out = durbin_watson(fit.resid)
    
    for col, val in zip(acc_df.columns, out):
        print(adjust(col), ':', round(val, 2))
    cointegration_test(acc_df, key_col)
    for name in fit.names:
        j = fit.get_eq_index(name)
        lookup[j] = name

    for i in range(len(fit.coefs[0])):
        row = lookup[i]
        for j in range(len(fit.coefs)):
            t = j+1
            row_name = "{} t-{}".format(row, t)
            pval = fit.pvalues[key_col]["L{}.{}".format(t, row)]
            res = ""
            if pval <= 0.10 and pval > 0.05:
                res = "{:.4f}*".format(fit.coefs[j][index][i])
            elif pval <= 0.05 and pval > 0.01:
                res = "{:.4f}**".format(fit.coefs[j][index][i])
            elif pval <= 0.01:
                res = "{:.4f}***".format(fit.coefs[j][index][i])
            else:
                res = fit.coefs[j][index][i]
            df_out.loc[row_name] = res
    df = pd.concat([df, df_out], axis=1, sort=False)
    counter += 1
    if counter == stop:
        print(fit.summary())
        break
df = df.fillna("-")
df = df.replace()
df

/home/kate/final year project/sentiment_analysis/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:218: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

/home/kate/final year project/sentiment_analysis/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:218: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.



UK Negative : 2.01
FTSE 100 Return : 2.0

Name   ::  Test Stat > C(95%)    =>   Signif  
 ----------------------------------------
UK Negative ::  218.62    > 12.3212   =>   True
UK Negative : 2.01
FTSE 100 Return : 2.0
ISEQ All Share Return : 1.99

Name   ::  Test Stat > C(95%)    =>   Signif  
 ----------------------------------------
UK Negative ::  365.18    > 24.2761   =>   True
UK Negative : 2.0
FTSE 100 Return : 2.0
ISEQ All Share Return : 1.99
Euro Stoxx 50 Volume : 2.0

Name   ::  Test Stat > C(95%)    =>   Signif  
 ----------------------------------------
UK Negative ::  421.54    > 40.1749   =>   True


/home/kate/final year project/sentiment_analysis/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:218: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.



,Model 1,Model 2,Model 3
UK Negative t-1,0.3541***,0.3386***,0.3345***
UK Negative t-2,0.0838**,0.0737**,0.0766**
UK Negative t-3,0.1158***,0.1265***,0.1238***
UK Negative t-4,0.1459***,0.1541***,0.1549***
UK Negative t-5,0.0814**,0.0897***,0.0919***
FTSE 100 Return t-1,-0.0405791,0.112766,0.1313*
FTSE 100 Return t-2,-0.0928091,-0.0130668,0.00361842
FTSE 100 Return t-3,-0.0429673,-0.103553,-0.0892889
FTSE 100 Return t-4,0.0500612,0.0487086,0.0681868
FTSE 100 Return t-5,0.0361706,-0.0890764,-0.0805872


In [20]:
h1 = fit.test_causality("ISEQ All Share Return", "UK Negative", )
h1.p

0.3282935671656886

In [7]:
spm = SentimentPriceModel()
spm.load_csv("w2v_vartest_1.csv")

,FTSE 100 Volume,ISEQ All Share Volume,Euro Stoxx 50 Volume,Word2Vec UK Negativ
date,,,,
2016-01-27,0.368897,0.338266,0.617111,-2.767022
2016-01-28,0.444651,0.771961,1.857351,-3.264444
2016-02-15,-0.285359,1.055438,1.819181,-1.742727
2016-02-19,-0.423521,1.931092,1.345190,-2.944776
2016-02-22,0.138648,2.633814,0.353910,3.980155
...,...,...,...,...
2019-12-17,1.332587,-0.571294,-0.389551,-0.390759
2019-12-18,-0.482754,-0.521523,-0.603189,-1.120399
2019-12-19,-0.558263,-0.615432,-0.481842,-2.086874


In [13]:
spm.save_to_database("swn actual 2", "second swn var test")

            FTSE 250 Volume  ISEQ All Share Volume  Euro Stoxx 50 Return  \
date                                                                       
2016-01-04        -0.846024               0.016210             -3.632469   
2016-01-05        -0.936379              -0.249442              0.168752   
2016-01-06        -0.403054              -0.011577             -1.150151   
2016-01-07        -0.300610               0.617958             -1.703234   
2016-01-08        -0.864265               0.633654             -2.036661   

            Euro Stoxx 50 Volume  The Guardian(London) Negative  
date                                                             
2016-01-04              0.792579                            NaN  
2016-01-05             -0.150276                            NaN  
2016-01-06              0.503741                            NaN  
2016-01-07              1.891533                            NaN  
2016-01-08              0.878035                            NaN  
Lengt

INFO - 00:12:40: Created 5036/5036 required objects during save


In [7]:
word2vec = gensim.models.Word2Vec.load("word2vec.model")
vectors = word2vec.wv
cat = "Negativ"
sw = stopwords.words('english')
topn=1

INFO - 21:49:51: loading Word2Vec object from word2vec.model
INFO - 21:49:51: loading wv recursively from word2vec.model.wv.* with mmap=None
INFO - 21:49:51: setting ignored attribute vectors_norm to None
INFO - 21:49:51: loading vocabulary recursively from word2vec.model.vocabulary.* with mmap=None
INFO - 21:49:51: loading trainables recursively from word2vec.model.trainables.* with mmap=None
INFO - 21:49:51: setting ignored attribute cum_table to None
INFO - 21:49:51: loaded word2vec.model


In [11]:
fit.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Mon, 27, Apr, 2020
Time:                     23:40:48
--------------------------------------------------------------------
No. of Equations:         4.00000    BIC:                  0.0238401
Nobs:                     952.000    HQIC:                 -0.190986
Log likelihood:          -5181.47    FPE:                   0.723840
AIC:                    -0.323200    Det(Omega_mle):        0.674365
--------------------------------------------------------------------
Results for equation FTSE 100 Return
                              coefficient       std. error           t-stat            prob
-------------------------------------------------------------------------------------------
const                            0.012948         0.030846            0.420           0.675
L1.FTSE 100 Return              -0.024022         0.041492           -0.579           0.563
L1.ISE

In [25]:
fit.coefs_exog

array([[ 0.01294837],
       [ 0.03866326],
       [-0.00854698],
       [ 0.02580169]])

In [8]:
cat = "Negativ"
words = [w.lower() for w in Category.objects.get(name=cat).words.all().values_list('word', flat=True)]
expanded_list = set()
index = 0
length = len(words)
for w in words:
    index +=1
    try:
        if w in sw:
            continue
        werds = [x for (x,_) in vectors.most_similar(positive=w, topn=topn)]
        for x in werds:
            expanded_list.add(x)
            progress(index, length, status="Added word")
         
    except Exception as e:
        progress(index, length, status="Word not added. {}".format(e))
    expanded_list.add(w)

INFO - 21:49:55: precomputing L2-norms of word weight vectors


In [13]:
expanded_list

{'shameful',
 'severity',
 'feature',
 'costly',
 'jeopardy',
 'arrest',
 'odd',
 'horrible',
 'narcissist',
 'condemnation',
 'reich',
 'disproportionately',
 'pigeon',
 'jumpy',
 'agile',
 'worse',
 'chiefly',
 'humiliation',
 'pessimistic',
 'deflect',
 'exile',
 'unfair',
 'deceptive',
 'murderous',
 'retire',
 'furnace',
 'disarm',
 'demand',
 'vacillation',
 'lost',
 'questionable',
 'britpop',
 'casualty',
 'pollutant',
 'fatigue',
 'exempt',
 'bombast',
 'allege',
 'slap',
 'shred',
 'invade',
 'clamorous',
 'controversial',
 'tutorial',
 'shamble',
 'belie',
 'languish',
 'front',
 'destroy',
 'profess',
 'independence',
 'suez',
 'lonely',
 'alienate',
 'egregious',
 'disclaim',
 'disregard',
 'esoteric',
 'context',
 'dissention',
 'aggression',
 'mess',
 'fire',
 'tedious',
 'backstabbing',
 'administrative_burden',
 'deviation',
 'primitive',
 'aghast',
 'counteract',
 'chore',
 'chide',
 'allegation',
 'thud',
 'bite',
 'illiterate',
 'erase',
 'chastise',
 'reality',
 'r

In [ ]:
from 